In [1]:
import sys
sys.path.append('../')


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report

from src.model.functions.transformations import get_delay_feature, get_dummies_features
from src.model.functions.validations import check_missing_features, check_features_for_nan, check_datetime_format

warnings.filterwarnings('ignore')

data = pd.read_csv('../../data/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha-I    68206 non-null  object
 1   Vlo-I      68206 non-null  object
 2   Ori-I      68206 non-null  object
 3   Des-I      68206 non-null  object
 4   Emp-I      68206 non-null  object
 5   Fecha-O    68206 non-null  object
 6   Vlo-O      68205 non-null  object
 7   Ori-O      68206 non-null  object
 8   Des-O      68206 non-null  object
 9   Emp-O      68206 non-null  object
 10  DIA        68206 non-null  int64 
 11  MES        68206 non-null  int64 
 12  AÑO        68206 non-null  int64 
 13  DIANOM     68206 non-null  object
 14  TIPOVUELO  68206 non-null  object
 15  OPERA      68206 non-null  object
 16  SIGLAORI   68206 non-null  object
 17  SIGLADES   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


## Preprocess - Adaptations

In [13]:
threshold_in_minutes = 15
categorical_features = ['OPERA', 'TIPOVUELO', 'MES']

get_delay_feature(data, threshold_in_minutes)

features = get_dummies_features(data, categorical_features)
target = get_delay_feature(data, threshold_in_minutes)

top_10_features = [
    "OPERA_Latin American Wings", 
    "MES_7",
    "MES_10",
    "OPERA_Grupo LATAM",
    "MES_12",
    "TIPOVUELO_I",
    "MES_4",
    "MES_11",
    "OPERA_Sky Airline",
    "OPERA_Copa Air"
]

x_train, x_test, y_train, y_test = train_test_split(features, target['delay'], test_size = 0.33, random_state = 42)

n_y0 = len(y_train[y_train == 0])
n_y1 = len(y_train[y_train == 1])
scale = n_y0/n_y1
print(scale)

reg_model_2 = LogisticRegression(class_weight={1: n_y0/len(y_train), 0: n_y1/len(y_train)})
reg_model_2.fit(x_train, y_train)

y_preds_2 = reg_model_2.predict(x_test)
print(classification_report(y_test, y_preds_2))

4.4402380952380955
              precision    recall  f1-score   support

           0       0.88      0.60      0.71     18294
           1       0.27      0.65      0.38      4214

    accuracy                           0.61     22508
   macro avg       0.58      0.62      0.55     22508
weighted avg       0.77      0.61      0.65     22508



In [11]:
#check_missing_features(data[['TIPOVUELO', 'MES']],categorical_features)

data['TIPOVUELO'] = np.nan

#check_features_for_nan(data[categorical_features],categorical_features)

In [12]:
dates_features = ['TIPOVUELO', 'Fecha-I']
#check_datetime_format(data, dates_features, datetime_format="%Y-%m-%d %H:%M:%S")

## Fit - Adaptations

In [18]:
import xgboost as xgb

top_n = 10

xgb_model = xgb.XGBClassifier(random_state=1, learning_rate=0.01)
xgb_model.fit(x_train, y_train)

importance = xgb_model.get_booster().get_score(importance_type='gain')

sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

top_10_features = []

for feature, score in sorted_importance[:top_n]:
    top_10_features.append(feature)



TIPOVUELO_I
OPERA_Copa Air
MES_12
OPERA_Air Canada
OPERA_Qantas Airways
MES_7
OPERA_Gol Trans
OPERA_American Airlines
OPERA_Aeromexico
OPERA_Delta Air
MES_11
OPERA_United Airlines
OPERA_Sky Airline
OPERA_Latin American Wings
MES_10
MES_3
MES_4
OPERA_Aerolineas Argentinas
MES_9
MES_1
MES_2
MES_6
OPERA_Avianca
OPERA_Grupo LATAM
MES_8
OPERA_JetSmart SPA


## Model Class

In [9]:
from model import DelayModel

model = DelayModel()

features, outcome = model.preprocess(data, 'delay')

In [13]:
model.fit(features, outcome)

In [11]:
model.preprocess(pd.DataFrame())

Error: The following features are missing from the DataFrame: ['OPERA', 'TIPOVUELO', 'MES']


In [14]:
outcome = model.predict(features)

In [35]:
# Define the single entry as a dictionary
flight_data = {
    "OPERA": "Aerolineas Argentinas", 
    "TIPOVUELO": "N", 
    "MES": 3
}

# Create a DataFrame from the dictionary
df = pd.DataFrame([flight_data])

testing_features = model.preprocess(df)


Error: "None of [Index(['OPERA_Latin American Wings', 'MES_7', 'MES_10', 'OPERA_Grupo LATAM',\n       'MES_12', 'TIPOVUELO_I', 'MES_4', 'MES_11', 'OPERA_Sky Airline',\n       'OPERA_Copa Air'],\n      dtype='object')] are in the [columns]"


In [31]:
sum(outcome)/len(outcome)

0.5193824590212005